In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [5]:
test_data.shape

(125690, 12)

In [6]:
# Clean Title means if the car has been crashed or not and it creates a new column where 1 is clean and 0 is crashed
train_data["clean_title_bool"] = train_data["clean_title"].map({"Yes":1, "No": 0})
test_data["clean_title_bool"] = test_data["clean_title"].map({"Yes":1, "No": 0})

In [7]:
train_data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,clean_title_bool
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,1.0
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,1.0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,1.0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,1.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,1.0


In [8]:
test_data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,clean_title_bool
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes,1.0
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes,1.0
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,1.0


## Preprocessing

In [9]:
# Drop unnecessary columns
train_data = train_data.drop(columns=["clean_title", "clean_title_bool"])
test_data = test_data.drop(columns=["clean_title", "clean_title_bool"])

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 17.3+ MB


In [11]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
dtypes: int64(3), object(8)
memory usage: 10.5+ MB


In [12]:
le = LabelEncoder()

## Brand

In [13]:
# Create a new colum with the label of the car
train_data["encoded_brand"] = le.fit_transform(train_data["brand"])
test_data["encoded_brand"] = le.fit_transform(test_data["brand"])

In [14]:
# Shows the brand names and how many there are 
train_data["brand"].unique(), train_data["brand"].nunique()

(array(['MINI', 'Lincoln', 'Chevrolet', 'Genesis', 'Mercedes-Benz', 'Audi',
        'Ford', 'BMW', 'Tesla', 'Cadillac', 'Land', 'GMC', 'Toyota',
        'Hyundai', 'Volvo', 'Volkswagen', 'Buick', 'Rivian', 'RAM',
        'Hummer', 'Alfa', 'INFINITI', 'Jeep', 'Porsche', 'McLaren',
        'Honda', 'Lexus', 'Dodge', 'Nissan', 'Jaguar', 'Acura', 'Kia',
        'Mitsubishi', 'Rolls-Royce', 'Maserati', 'Pontiac', 'Saturn',
        'Bentley', 'Mazda', 'Subaru', 'Ferrari', 'Aston', 'Lamborghini',
        'Chrysler', 'Lucid', 'Lotus', 'Scion', 'smart', 'Karma',
        'Plymouth', 'Suzuki', 'FIAT', 'Saab', 'Bugatti', 'Mercury',
        'Polestar', 'Maybach'], dtype=object),
 57)

## Model

In [15]:
train_data["encoded_model"] = le.fit_transform(train_data["model"])
test_data["encoded_model"] = le.fit_transform(test_data["model"])

In [16]:
train_data["model"].nunique()

1897

In [17]:
models = train_data["model"].unique().tolist()
print(models)

['Cooper S Base', 'LS V8', 'Silverado 2500 LT', 'G90 5.0 Ultimate', 'Metris Base', 'A6 2.0T Sport', 'A8 L 3.0T', 'Silverado 1500 1LZ', 'F-150 XLT', 'M4 Base', 'Camaro 1LT', 'Model S P100D', 'Escalade ESV Platinum', 'S4 3.0T Premium Plus', 'Rover Range Rover P530 SE SWB', 'AMG C 63 S', 'Yukon Denali', 'Rover Defender SE', 'Tundra SR5', 'AMG C 63 Base', 'Rover Defender S', 'Equus Signature', 'Mustang Premium', 'A8 L 55', 'XC70 T6 Platinum', 'Sequoia Limited', 'F-250 XLT', 'Tacoma PreRunner', 'A5 2.0T Premium', 'A3 2.0T Premium', 'E-Class E 300', 'E-Class E 350 4MATIC', 'Jetta S', '528 i xDrive', '330 i xDrive', 'AMG G 63 Base', 'C-Class C 300 4MATIC Sport', 'Enclave Premium', '328 xi', 'Corvette Grand Sport', 'AMG GT 53 Base', 'R1S Adventure Package', '1500 Laramie', 'Corvette Base', 'H2 Base', 'Romeo Stelvio Ti Sport', 'QX60 Base', 'Model 3 Long Range', 'Model X 75D', 'Genesis Coupe 3.8 Base', 'M6 Base', 'Gladiator Rubicon', 'Thunderbird Deluxe', 'Rover Range Rover Sport HSE', 'SL-Class

## Fuel Type

In [18]:
train_data["fuel_type"].unique(), train_data["fuel_type"].nunique()

(array(['Gasoline', 'E85 Flex Fuel', nan, 'Hybrid', 'Diesel',
        'Plug-In Hybrid', '–', 'not supported'], dtype=object),
 7)

In [19]:
nan_count_fuel_type = train_data['fuel_type'].isna().sum()
print(f"Number of NaN values in 'fuel_type': {nan_count_fuel_type}")

Number of NaN values in 'fuel_type': 5083


In [20]:
# Replace problematic values with 'Unknown' or np.nan
train_data['fuel_type'] = train_data['fuel_type'].replace(
    {'–': 'Unknown', 'not supported': 'Unknown'}
)

# Optionally fill NaN with 'Unknown' or drop them
train_data['fuel_type'] = train_data['fuel_type'].fillna('Unknown')  # or .dropna(subset=['fuel_type'])

In [21]:
train_data["fuel_type"].unique()

array(['Gasoline', 'E85 Flex Fuel', 'Unknown', 'Hybrid', 'Diesel',
       'Plug-In Hybrid'], dtype=object)

In [22]:
train_data["encoded_fuel_type"] = le.fit_transform(train_data["fuel_type"])
test_data["encoded_fuel_type"] = le.fit_transform(test_data["fuel_type"])

## Transmission

In [23]:
# Define a mapping for standardizing transmission types
transmission_mapping = {
    # Automatic transmissions
    'A/T': 'Automatic', '8-Speed A/T': 'Automatic', '7-Speed A/T': 'Automatic',
    '10-Speed Automatic': 'Automatic', '1-Speed A/T': 'Automatic',
    '6-Speed A/T': 'Automatic', '9-Speed A/T': 'Automatic',
    'Automatic': 'Automatic', 'CVT Transmission': 'CVT',
    'Automatic CVT': 'CVT', '8-Speed Automatic': 'Automatic',
    '6-Speed Automatic': 'Automatic', '7-Speed Automatic': 'Automatic',
    '5-Speed A/T': 'Automatic', '4-Speed Automatic': 'Automatic',
    '2-Speed A/T': 'Automatic', '10-Speed A/T': 'Automatic',
    'Transmission w/Dual Shift Mode': 'Automatic',

    # Manual transmissions
    'M/T': 'Manual', '5-Speed M/T': 'Manual', '6-Speed M/T': 'Manual',
    '7-Speed M/T': 'Manual', '6-Speed Manual': 'Manual',
    '8-Speed Manual': 'Manual', '7-Speed Manual': 'Manual',
    '6 Speed Mt': 'Manual',

    # CVT
    'CVT-F': 'CVT',

    # Remove invalid or unclear values
    '–': 'Unknown', '2': 'Unknown', 'SCHEDULED FOR OR IN PRODUCTION': 'Unknown', 'F': 'Unknown',
    'Variable': 'Unknown', 'Transmission Overdrive Switch': 'Unknown'
    }

# Apply mapping to 'transmission' column
train_data['transmission'] = train_data['transmission'].map(transmission_mapping)

# remove nan inside 'transmission' column
train_data['transmission'] = train_data['transmission'].fillna('Unknown')

In [24]:
train_data["transmission"].unique(), train_data["transmission"].nunique()

(array(['Automatic', 'Unknown', 'CVT', 'Manual'], dtype=object), 4)

In [25]:
train_data["encoded_transmission"] = le.fit_transform(train_data["transmission"])
test_data["encoded_transmission"] = le.fit_transform(test_data["transmission"])

In [26]:
train_data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,encoded_brand,encoded_model,encoded_fuel_type,encoded_transmission
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,Automatic,Yellow,Gray,None reported,4200,31,495,2,0
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,Automatic,Silver,Beige,At least 1 accident or damage reported,4999,28,930,2,0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,Automatic,Blue,Gray,None reported,13900,9,1575,1,0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Automatic,Black,Black,None reported,45000,16,758,2,0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Automatic,Black,Beige,None reported,97500,36,1077,2,0


## Feature Engineering

In [27]:
current_year = 2025
# Calculate vehicle age and mileage per year
def create_features(DataFrame):
    DataFrame["vehicle_age"] = current_year - DataFrame["model_year"]  
    DataFrame["milage_per_year"] = DataFrame["milage"] / DataFrame["vehicle_age"]
    DataFrame["milage_per_year"] = DataFrame["milage_per_year"].replace([np.inf, -np.inf], 0)  
    DataFrame["has_accident_history"] = DataFrame["accident"].apply(lambda x: 1 if pd.isnull(x) or x == 'At least 1 accident or damage reported' else 0)
    return DataFrame
train_data = create_features(train_data)
test_data = create_features(test_data)

In [28]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    188533 non-null  int64  
 1   brand                 188533 non-null  object 
 2   model                 188533 non-null  object 
 3   model_year            188533 non-null  int64  
 4   milage                188533 non-null  int64  
 5   fuel_type             188533 non-null  object 
 6   engine                188533 non-null  object 
 7   transmission          188533 non-null  object 
 8   ext_col               188533 non-null  object 
 9   int_col               188533 non-null  object 
 10  accident              186081 non-null  object 
 11  price                 188533 non-null  int64  
 12  encoded_brand         188533 non-null  int64  
 13  encoded_model         188533 non-null  int64  
 14  encoded_fuel_type     188533 non-null  int64  
 15  

In [29]:
# Extract Horsepower (HP)
train_data["extract_horsepower"] = train_data["engine"].str.extract(r'(\d+(\.\d+)?)HP')[0].astype(float)
test_data["extract_horsepower"] = test_data["engine"].str.extract(r'(\d+(\.\d+)?)HP')[0].astype(float)

# Extract Displacement
train_data["extract_displacement"] = train_data["engine"].str.extract(r'(\d+\.\d+)L|(\d+\.\d+) Liter').bfill(axis=1)[0].astype(float)
test_data["extract_displacement"] = test_data["engine"].str.extract(r'(\d+\.\d+)L|(\d+\.\d+) Liter').bfill(axis=1)[0].astype(float)

# Extract Cylinder Count
train_data["extract_cylinder_count"] = train_data["engine"].str.extract(r'(\d+) Cylinder')[0].astype(float)
test_data["extract_cylinder_count"] = test_data["engine"].str.extract(r'(\d+) Cylinder')[0].astype(float)

# Extract Fuel Type
fuel_types = ['Gasoline', 'Diesel', 'Electric', 'Hybrid', 'Flex Fuel']
train_data["extract_fuel_type"] = train_data["engine"].str.extract(f"({'|'.join(fuel_types)})")[0]
test_data["extract_fuel_type"] = test_data["engine"].str.extract(f"({'|'.join(fuel_types)})")[0]

In [30]:
# Impute Missing Values
imputer = KNNImputer(n_neighbors=3)
train_data[["extract_horsepower", "extract_displacement"]] = imputer.fit_transform(train_data[["extract_horsepower", "extract_displacement"]])
test_data[["extract_horsepower", "extract_displacement"]] = imputer.transform(test_data[["extract_horsepower", "extract_displacement"]])

In [31]:
train_data["extract_fuel_type"] = train_data["extract_fuel_type"].replace([np.nan, 'not supported', '–'], 'Unknown')
test_data["extract_fuel_type"] = test_data["extract_fuel_type"].replace([np.nan, 'not supported', '–'], 'Unknown')
train_data["extract_displacement"] = train_data["extract_displacement"].fillna(value=train_data["extract_displacement"].mean())
test_data["extract_displacement"] = test_data["extract_displacement"].fillna(value=test_data["extract_displacement"].mean())

In [32]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      188533 non-null  int64  
 1   brand                   188533 non-null  object 
 2   model                   188533 non-null  object 
 3   model_year              188533 non-null  int64  
 4   milage                  188533 non-null  int64  
 5   fuel_type               188533 non-null  object 
 6   engine                  188533 non-null  object 
 7   transmission            188533 non-null  object 
 8   ext_col                 188533 non-null  object 
 9   int_col                 188533 non-null  object 
 10  accident                186081 non-null  object 
 11  price                   188533 non-null  int64  
 12  encoded_brand           188533 non-null  int64  
 13  encoded_model           188533 non-null  int64  
 14  encoded_fuel_type   

## Feauture Selection

In [33]:
features = ["encoded_brand", "vehicle_age", "has_accident_history", "extract_horsepower"]

In [34]:
# This DataFrame is the data from the train.csv but only the selected features above ^ 
trainX = train_data[features]

In [35]:
# This DataFrame is the data from the test.csv but only the selected features above ^ 
testX = test_data[features]

In [36]:
# This is a list of prices taken from the column "price" inside the train.csv
trainY = train_data["price"]

In [37]:
# We are calling method `train_test_split` this method will split the trainX and trainY into many sections
# By changing the `test_size` parameter you change the percentage of the test data from the initial data(trainX, trainY)
# We are splitting the data into
# train_split_X and train_splitY  = 80% of the datasest
# test_split_X and test_splitY  = 20% of the datasest

train_split_X, test_split_X, train_split_Y, test_split_Y = train_test_split(trainX, trainY, test_size = 0.20, random_state = 32)

## Linear Regression Kaggle

In [38]:
model = LinearRegression()

In [39]:
model.fit(trainX, trainY)

LinearRegression()

In [40]:
linear_prediction = model.predict(testX)

In [41]:
linear_prediction[:10]

array([33388.05386849, 62841.20243969, 56656.54242526, 29250.86804695,
       38686.34445425, 31401.72629616, 20327.78182001, 45292.94050078,
       70740.12881848, 50937.71472   ])

In [42]:
output = pd.DataFrame({"id": test_data.id, "price": linear_prediction})
output.to_csv("submission.csv", index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


# Linear Regression (split)

In [43]:
model = LinearRegression()

In [44]:
model.fit(train_split_X, train_split_Y)

LinearRegression()

In [45]:
linear_prediction = model.predict(test_split_X)

In [46]:
accuracy = model.score(test_split_X, test_split_Y)

In [47]:
accuracy

0.059291337257780796

In [48]:
rms = mean_squared_error(test_split_Y, linear_prediction)
rms

7744468207.987552

In [49]:
from sklearn.metrics import mean_absolute_error
m = mean_absolute_error(test_split_Y, linear_prediction)
m

24037.87229208987

## Random Forest Regresson Kaggle

In [51]:
model = RandomForestRegressor()

In [52]:
model.fit(trainX, trainY)

RandomForestRegressor()

In [53]:
random_forest_prediction = model.predict(testX)

In [54]:
random_forest_prediction[:10]

array([27855.60379261, 66824.36460861, 72518.87969293, 24872.96248554,
       30677.54348747, 18716.57614067, 13230.44573784, 25211.24369048,
       61738.70529061, 28387.06830546])

In [55]:
output = pd.DataFrame({"id": test_data.id, "price": random_forest_prediction})
output.to_csv("submission.csv", index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


## Random Forest Regresson Split

In [56]:
model = RandomForestRegressor()

In [57]:
model.fit(train_split_X, train_split_Y)

RandomForestRegressor()

In [58]:
random_forest_prediction = model.predict(test_split_X)

In [59]:
accuracy = model.score(test_split_X, test_split_Y)

In [60]:
accuracy

0.02421998286202942

In [61]:
rms = mean_squared_error(test_split_Y, random_forest_prediction)
rms

8033196270.018155

In [62]:
from sklearn.metrics import mean_absolute_error
m = mean_absolute_error(test_split_Y, random_forest_prediction)
m

24037.87229208987